In [1406]:
import pandas as pd
import re
from collections import defaultdict
from time import time

start_time = time()

In [1407]:
# 필요 경로들
input_file_path = "../asset/ohw4.csv"
conform_path = "../asset/conform4.csv"
related_path = "../asset/related4.csv"
id_path = "../asset/ohw_id.csv"
id_mean_path = '../asset/ohw_id_mean.csv'
id_exam_path = '../asset/ohw_id_exam.csv'
id_use_path = '../asset/ohw_id_use.csv'
id_rel_path = '../asset/ohw_id_rel.csv'
id_present_path = '../asset/id_present.csv'
output_file_path = "../result/result4_0119.csv"

In [1408]:
# 수정할 파일 경로 
target = pd.read_csv(input_file_path)
target = target.fillna('')
target.columns

Index(['제공처ID', '제공처표제어', '어깨번호', '표제어종류', '형태', '용법 정보', '원어', '어원', '한글 자소',
       '상위품사', '뜻번호', '하위품사(상)', '하위품사(하)', '뜻풀이', '전문분야', '패턴', '의미정보', '꼴정보',
       '패턴정보', '예문'],
      dtype='object')

In [1409]:
target['표제어'] = target['제공처표제어']
target['노출용 표제어'] = target['제공처표제어']
target['엔트리ID'] = ''
target['집필 여부'] = int(0)
target['집필상태'] = '서비스 반영'
target['편집자'] = ''
target['최초 등록 시간'] = ''
target['마지막 편집 시간'] = ''
target['엔트리ID'] = ''
target['원어맴버ID'] = ''
target['고유어 여부'] = 'NO'
target['한자어 여부'] = 'NO'
target['외래어 여부'] = 'NO'
target['고사성어 여부'] = 'NO'
target['북한어 여부'] = 'NO'
target['옛말 여부'] = 'NO'
target['방언 여부'] = 'NO'
target['혼종어 여부'] = 'NO'
target['서비스 노출(어원부)'] = 'YES' #어원에 값이 있으면 YES
target['검색 노출(어원부)'] = 'NO'
target['발음ID'] = ''
target['발음종류'] = ''
target['발음기호(발음부)'] = ''
target['노출용 발음 기호'] = ''
target['발음 파일'] = ''
target['별명ID(이형태)'] = ''
target['종류(이형태)'] = ''
target['이형태'] = ''
target['노출용 이형태'] = ''
target['중요도'] = '일반'
target['오늘의 단어 선정 여부'] = 'NO'
target['참고 원어 설명'] = ''
target['혁명정보'] = ''
target['역사정보'] = ''
target['긴급 대응'] = ''
target['서비스 노출(표제어부)'] = 'YES'
target['검색 노출(표제어부)'] = '통검&사전'
target['서비스 노출(품사부)'] = 'YES' # 표제어있는행 + 품사있는 행 아니면 ''
target['검색 노출(품사부)'] = '통검&사전' # 표제어있는행 + 품사있는 행 아니면 ''
target['품사 앞 설명'] = ''
target['특수 뜻풀이'] = ''
target['노출용 뜻풀이'] = ''
target['화용표지'] = ''
target['차별표현'] = ''
target['차별표현 링크'] = ''
target['뜻풀이ID'] = ''
target['뜻풀이 유형'] = '일반형'
target['서술형 뜻풀이'] = ''
target['특수 뜻풀이 타입'] = ''
target['규범 정보'] = ''
target['뜻풀이 방언'] = ''
target['참고 문법'] = ''
target['서비스 노출(뜻풀이부)'] = 'YES' # 패턴설명 없으면 ''
target['검색 노출(뜻풀이부)'] = '통검&사전' # 패턴설명 없으면 ''
target['노출용 예문'] = ''
target['예문ID'] = ''
target['예문 종류'] = '' # 공백
target['연관어'] = '' # 공백
target['예문 출처'] = '' # 공백
target['출처타입'] = 'CP' 
target['서비스 노출(예문부)'] = 'YES'
target['검색 노출(예문부)'] = '통검&사전'
target['연관단어ID'] = ''
target['출처'] = 'KOREA' # 없으면 ''
target['연관어뜻ID'] = ''
target['상세페이지연결여부'] = 'YES' # 없으면 NO
target['서비스 노출(연관어부)'] = 'YES' # 없으면 NO
target['검색 노출(연관어부)'] = '통검&사전'
target['활용형ID'] = ''
target['대분류'] = '활용형'
target['중분류'] = ''
target['대분류 노출 여부'] = 'Yes'
target['중분류 노출 여부'] = 'No'
target['소분류 노출 여부'] = 'No'
target['우선순위'] = int(2)
target['발음기호'] = ''
target['발음파일'] = ''
target['연관단어ID(연관단어부)'] = ''
target['참고 문법(연관단어부)'] = ''
target['출처(연관단어부)'] = 'KOREA'
target['상세페이지연결여부(연관단어부)'] = 'YES'
target['서비스 노출(연관단어부)'] = 'YES'
target['검색 노출(연관단어부)'] = '통검&사전'
target['연관어(예문)'] = ''
target['종류'] = ''
target['노출용 연관어'] = ''
target['연관어표제어ID'] = ''
target['활용형'] = ''
target['부가명칭'] = ''
target['부가설명'] = ''
target['종류(연관어부)'] = ''
target['연관어(연관어부)'] = ''
target['노출용 연관어(연관어부)'] = ''
target['연관어표제어ID(연관어부)'] = ''

In [1410]:
target = target[[
    # 표제어부
    '엔트리ID', '집필 여부', '표제어', '어깨번호', '집필상태', '편집자', '최초 등록 시간', '마지막 편집 시간', '표제어종류', '제공처ID', '제공처표제어', '원어맴버ID', '노출용 표제어',
    '형태', '용법 정보', '원어', '고유어 여부', '한자어 여부', '외래어 여부', '고사성어 여부', '북한어 여부', '옛말 여부', '방언 여부', '혼종어 여부', '어원', '서비스 노출(어원부)', 
    '검색 노출(어원부)', '발음ID', '발음종류', '발음기호(발음부)', '노출용 발음 기호', '발음 파일', '별명ID(이형태)', '종류(이형태)', '이형태', '노출용 이형태', '중요도', '오늘의 단어 선정 여부', 
    '한글 자소', '참고 원어 설명', '혁명정보', '역사정보', '긴급 대응', '서비스 노출(표제어부)', '검색 노출(표제어부)',
    # 품사부 & 뜻풀이부
    '서비스 노출(품사부)', '검색 노출(품사부)', '상위품사', '품사 앞 설명', '뜻번호', '하위품사(상)', '하위품사(하)', '특수 뜻풀이', '뜻풀이', '노출용 뜻풀이', '화용표지', '전문분야', '차별표현', '차별표현 링크', '뜻풀이ID',
    '뜻풀이 유형', '서술형 뜻풀이', '특수 뜻풀이 타입', '규범 정보', '뜻풀이 방언', '참고 문법', '패턴', '의미정보', '꼴정보', '패턴정보', '서비스 노출(뜻풀이부)', '검색 노출(뜻풀이부)',
    # 예문부
    '예문', '노출용 예문', '예문ID', '예문 종류', '연관어(예문)', '예문 출처', '출처타입', '서비스 노출(예문부)', '검색 노출(예문부)', 
    # 연관단어부
    '연관단어ID', '종류', '연관어', '노출용 연관어', '출처', '연관어뜻ID', '연관어표제어ID', '상세페이지연결여부', '서비스 노출(연관어부)', '검색 노출(연관어부)', 
    # 활용형부 
    '활용형ID', '대분류', '중분류', '대분류 노출 여부', '중분류 노출 여부', '소분류 노출 여부', '우선순위', '활용형', '발음기호', '부가명칭', '부가설명',
    # 연관단어부 (전체)
    '연관단어ID(연관단어부)', '종류(연관어부)', '연관어(연관어부)', '노출용 연관어(연관어부)', '참고 문법(연관단어부)', '출처(연관단어부)', '연관어표제어ID(연관어부)', 
    '상세페이지연결여부(연관단어부)', '서비스 노출(연관단어부)', '검색 노출(연관단어부)'
]]

In [1411]:
# 불필요한 괄호 제거 
target['형태'] = target['형태'].str.replace('[', '').str.replace(']', '')
target['원어'] = target['원어'].str.replace('(', '').str.replace(')', '')
target['패턴'] = target['패턴'].str.replace('◀', '').str.replace('▶', '')
target['전문분야'] = target['전문분야'].str.replace('｢', '').str.replace('｣', '') 

In [1412]:
# 품사 정보 만들기
suppos_format = {'ꃃ':'명사',  'ꂽ' : '대명사', 'ꃓ' : '수사',  'ꂿ' : '동사', 'ꃰ' : '형용사', 
            'ꂴ' : '관형사', 'ꃌ' : '부사', 'ꂳ' : '감탄사', 'ꃦ' : '조사', 'ꃥ' : '접속사', 'ꃆ' : ['접두사', '접미사'], 
            'ꃁ' : ['종결 어미', '연결 어미', '전성 어미', '선어말 어미', '어미']}
subpos_format = {'무', '일반', '고유', '자립', '의존', '인칭', '지시', '타', '자', '보', '활', '두', '미', '선', '종', '연', '잔'}

In [1413]:
target = target.sort_values(by=['제공처ID', '어깨번호', '뜻번호']).reset_index(drop=True)

In [1414]:
# 품사 폰트 변경
LEN = len(target)
for i in range(LEN):
    if target.loc[i, '하위품사(상)'] == '' or target.loc[i, '하위품사(상)'] == '무':
        continue
    suppos = target.loc[i, '하위품사(상)']
    if suppos not in suppos_format:
        continue
    # 명사
    if suppos == 'ꃃ':
        if target.loc[i, '하위품사(하)'] == '무':
            target.loc[i, '하위품사(상)'] = '명사'
        else:
            target.loc[i, '하위품사(상)'] = target.loc[i, '하위품사(하)'] + '명사'
    # 대명사
    elif suppos == 'ꂽ':
        if target.loc[i, '하위품사(하)'] == '무':
            target.loc[i, '하위품사(상)'] = '대명사'
        else:
            target.loc[i, '하위품사(상)'] = target.loc[i, '하위품사(하)'] + " " + '대명사'
    # 접사
    elif suppos == 'ꃆ':
        if target.loc[i, '하위품사(하)'] == '두':
            target.loc[i, '하위품사(상)'] = '접두사'
        else: 
            target.loc[i, '하위품사(상)'] = '접미사'
    # 동사
    elif suppos == 'ꂿ':
        if target.loc[i, '하위품사(하)'] == '무':
            target.loc[i, '하위품사(상)'] = '동사'
        elif target.loc[i, '하위품사(하)'] == '보':
            target.loc[i, '하위품사(상)'] = '보조동사'
        else: 
            target.loc[i, '하위품사(상)'] = target.loc[i, '하위품사(하)'] + "동사"
    # 형용사 
    elif suppos == 'ꃰ':
        if target.loc[i, '하위품사(하)'] == '무':
            target.loc[i, '하위품사(상)'] = '형용사'
        else:
            target.loc[i, '하위품사(상)'] = '보조형용사'
    # 어미 
    elif suppos == 'ꃁ':
        if target.loc[i, '하위품사(하)'] == '무':
            target.loc[i, '하위품사(상)'] = '어미'
        elif target.loc[i, '하위품사(하)'] == '선':
            target.loc[i, '하위품사(상)'] = '선어말 어미'
        elif target.loc[i, '하위품사(하)'] == '종':
            target.loc[i, '하위품사(상)'] = '종결 어미'
        elif target.loc[i, '하위품사(하)'] == '연':
            target.loc[i, '하위품사(상)'] = '연결 어미'
        elif target.loc[i, '하위품사(하)'] == '전':
            target.loc[i, '하위품사(상)'] = '전성 어미'
    # 준 
    elif target.loc[i, '하위품사(상)'] == '준':
        target.loc[i, '하위품사(상)'] = ''
    # 나머지 
    else:
        target.loc[i, '하위품사(상)'] = suppos_format[suppos]

target = target.drop(columns=['하위품사(하)'])
target['하위품사(상)'] = target['하위품사(상)'].str.replace('무', '')
target['상위품사'] = target['하위품사(상)']
target = target.rename(columns={'하위품사(상)' : '하위품사'})

In [1415]:
# def reset_meaning_and_fill_subpos(group: pd.DataFrame) -> pd.DataFrame:
#     """
#     같은 제공처ID 그룹 내에서 순서대로 보며:
#         1) 이전 행과 상위품사가 같으면 뜻번호 += 1
#         2) 달라지면 뜻번호를 1로 reset
#         3) 하위품사 = 상위품사 로 동기화
#     """
#     # 필요하다면 특정 정렬 기준이 있으면 sort_values() 사용
#     # group = group.sort_values("어떤_칼럼")

#     group["상위품사"] = group["상위품사"].str.strip()

#     current_pos = None  # 현재 상위품사

#     for i in group.index:
#         upper_pos = group.loc[i, "상위품사"]

#         # (1) 만약 상위품사가 이전과 다르면 새 그룹 시작
#         if upper_pos != current_pos:
#             current_pos = upper_pos
#             counter = 1
#         else:
#             # (2) 이전과 같은 상위품사라면 뜻번호 증가
#             counter += 1

#         # (3) 뜻번호 갱신
#         group.loc[i, "뜻번호"] = counter

#         # (4) 하위품사 = 상위품사 (원하시는 로직에 맞춰)
#         group.loc[i, "하위품사"] = upper_pos

#     return group

# target = target.groupby("제공처ID", group_keys=False).apply(reset_meaning_and_fill_subpos)

In [1416]:
# 뜻번호 수정
LEN = len(target)
current_eid = 0
for i in range(LEN):
    temp_eid = target.loc[i, '제공처ID']
    if temp_eid != current_eid:
        current_eid = temp_eid
        current_mean_num = 1
        current_low_pos = target.loc[i, '하위품사']
    else:
        if target.loc[i, '상위품사'] == '':
            target.loc[i, '하위품사'] = current_low_pos
            current_mean_num += 1
            target.loc[i, '뜻번호'] = current_mean_num
        else:
            current_mean_num = 1
            current_low_pos = target.loc[i, '하위품사']
            target.loc[i, '뜻번호'] = 1

# 1) 상위품사가 비어있지 않은 조건 만들기
#   - notna()로 결측치(NaN) 여부 확인
#   - 빈 문자열('')인 경우도 제외하려면 != '' 조건도 추가
cond_not_empty_pos = target["상위품사"].notna() & (target["상위품사"] != "")

# 2) 뜻번호가 1이 아닌 행
cond_not_1 = target["뜻번호"] != 1

# 3) 두 조건을 모두 만족하는 경우 → 뜻번호를 1로 설정
condition = cond_not_empty_pos & cond_not_1
target.loc[condition, "뜻번호"] = 1

In [1417]:
# 한글 자소 처리 
def just_first_letter(jamo):
    return jamo[0] if jamo[0] != '-' else jamo[1]

target['한글 자소'] = target['한글 자소'].apply(just_first_letter)

In [1418]:
# 폰트 변경 
pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
target['패턴'] = (
        target['패턴'].str.replace('ꄍ', '(명사)')
        .str.replace('ꄉ', '(동사)').str.replace('ꄺ', '(형용사)').str.replace('ꄖ', '(부사)')
)

In [1419]:
# 예문 분리하기 
target['예문'] = target['예문'].str.replace('¶', '')
target['예문'] = target['예문'].str.split(' / ')
target = target.explode('예문').reset_index(drop=True)

In [1420]:
# 예문의 ~ 변경
LEN = len(target)
for i in range(LEN):
    ohw = target.loc[i, '제공처표제어']
    template = '<b>' + ohw + '</b>'
    if '~' in target.loc[i, '예문']: 
        target.loc[i, '예문'] = target.loc[i, '예문'].replace('~', template)

In [1421]:
# 발음정보 relation, 연관어 relation 읽어오기 
conform = pd.read_csv(conform_path)
related = pd.read_csv(related_path)

conform = conform.fillna('')
related = related.fillna('')

In [1422]:
# 발음정보 relation 전처리 
conform['활용형'] = conform['활용형'].str.strip('<>')
conform['활용형'] = conform['활용형'].str.split(r'[/,]')
conform = conform.explode('활용형').reset_index(drop=True)
conform['활용형'] = conform['활용형'].str.strip()

In [1423]:
conform_name_format = {
    '여∃' : '여 불규칙',
    'ㅂ∃' : 'ㅂ 불규칙',
    'ㄷ∃' : 'ㄷ 불규칙',
    '르∃' : '르 불규칙',
    'ㅎ∃' : 'ㅎ 불규칙',
    'ㅅ∃' : 'ㅅ 불규칙'
}

conform_explan_format = {
    '여 불규칙' : '어미 \'-아\'가 \'-여\'로 변하는 불규칙 어미 활용',
    'ㅂ 불규칙' : '어간 끝음절의 받침 \'ㅂ\'이 \'아\'나 \'아\'로 시작하는 어미 앞에서 \'오\'로 변하고, \'어\'나 \'어\'로 시작되는 어미 및 매개 모음을 요구하는 어미 앞에서는 \'우\'로 변하는 불규칙 활용',
    'ㄷ 불규칙' : '어가 끝음절의 받침 \'ㄷ\'이 모음으로 된 어미 앞에서 \'ㄹ\'로 변하는 불규칙 활용',
    '르 불규칙' : '어간의 끝음절 \'르\'가 \'-아\'나 \'-어\' 앞에서 \'ㄹㄹ\'로 바뀌는 불규칙 활용',
    'ㅎ 불규칙' : '일부 형용사에서 어간 끝의 \'ㅎ\'이 어미 \'ㄴ\'이나 \'ㅁ\' 위에서 줄어 활용하는 형식',
    'ㅅ 불규칙' : '어간 끝음절의 받침 \'ㅅ\'이 모음으로 시작하는 어미 앞에서 탈락하는 활용 형식'
}

In [1424]:
conform['부가명칭'] = conform['부가명칭'].map(conform_name_format).fillna("")
conform['부가설명'] = conform['부가명칭'].map(conform_explan_format).fillna("")

In [1425]:
# def merge_target_conform(target: pd.DataFrame, 
#                         conform: pd.DataFrame) -> pd.DataFrame:
#     """
#     target과 conform을 '제공처ID' 기준으로 병합.
#     target 쪽 '제공처ID'가 여러 줄일 때,
#     conform에서 가져올 행 수가 적으면 위부터 붙이고 남는 target 행은 NaN.
#     conform의 행 수가 더 많으면 target 행 위에 붙이고 남은 conform 행 만큼 
#     새로 행 추가 (target 쪽 컬럼은 NaN).
#     """

#     # 병합할 새 컬럼들
#     merge_cols = ["활용형", "부가명칭", "부가설명"]

#     # 제공처ID별로 groupby
#     tg_group = target.groupby("제공처ID")
#     cf_group = conform.groupby("제공처ID")

#     # 모든 제공처ID의 집합
#     all_ids = sorted(set(target["제공처ID"]).union(set(conform["제공처ID"])))

#     # 결과를 모을 리스트
#     merged_list = []

#     for pid in all_ids:
#         # target 해당 그룹
#         if pid in tg_group.groups:
#             tg_df = tg_group.get_group(pid)
#         else:
#             # 해당 ID가 없으면 빈 df
#             tg_df = pd.DataFrame(columns=target.columns)

#         # conform 해당 그룹
#         if pid in cf_group.groups:
#             cf_df = cf_group.get_group(pid)
#         else:
#             cf_df = pd.DataFrame(columns=conform.columns)

#         T_count = len(tg_df)
#         C_count = len(cf_df)

#         # 두 그룹 중 더 긴 쪽의 길이
#         row_count = max(T_count, C_count)

#         # row_count 크기의 빈 DataFrame을 만들어서 기존 값 복사
#         # (target 쪽)
#         new_tg = pd.DataFrame(index=range(row_count), columns=target.columns)
#         if T_count > 0:
#             new_tg.iloc[:T_count] = tg_df.values
        
#         # (conform 쪽에서 가져올 열)
#         new_cf = pd.DataFrame(index=range(row_count), columns=merge_cols)
#         if C_count > 0:
#             new_cf.iloc[:C_count] = cf_df[merge_cols].values

#         # 나란히 붙이기
#         merged_df = pd.concat([new_tg.reset_index(drop=True),
#                                 new_cf.reset_index(drop=True)], axis=1)
#         merged_list.append(merged_df)

#     # 모든 ID 그룹을 다시 합침
#     final_df = pd.concat(merged_list, ignore_index=True)
#     return final_df

# # 머지 실행
# target = merge_target_conform(target, conform)
# target['활용형']   = target['활용형_2']
# target['부가명칭']  = target['부가명칭_2']
# target['부가설명']  = target['부가설명_2']

# target.drop(columns=['활용형_2','부가명칭_2','부가설명_2'], inplace=True)
# target.fillna('', inplace=True)

In [1426]:
def merge_target_conform(target: pd.DataFrame, 
                        conform: pd.DataFrame) -> pd.DataFrame:
    """
    target과 conform을 '제공처ID' 기준으로 병합하되,
    - '활용형', '부가명칭', '부가설명'만 conform에서 가져와서 target의 같은 칼럼에 덮어쓴다.
    - target 쪽 행 수 < conform 쪽 행 수이면, 남은 conform 행만큼 새 행을 추가
    - target 쪽 행 수 > conform 쪽 행 수이면, 남은 target 행은 그대로 둔다 (conform 값은 없음)
    """

    # 덮어쓸(merge할) 컬럼들
    merge_cols = ["활용형", "부가명칭", "부가설명"]

    # 그룹화
    tg_group = target.groupby("제공처ID")
    cf_group = conform.groupby("제공처ID")

    # 양쪽에 등장하는 모든 ID
    all_ids = sorted(set(target["제공처ID"]) | set(conform["제공처ID"]))

    merged_list = []

    for pid in all_ids:
        # target 중 pid 그룹
        if pid in tg_group.groups:
            tg_df = tg_group.get_group(pid).copy()
        else:
            # target에 해당 ID가 없다면 빈 DF
            tg_df = pd.DataFrame(columns=target.columns)

        # conform 중 pid 그룹
        if pid in cf_group.groups:
            cf_df = cf_group.get_group(pid).copy()
        else:
            cf_df = pd.DataFrame(columns=conform.columns)

        T_count = len(tg_df)
        C_count = len(cf_df)

        # 그룹 내에서 필요한 최대 행 개수
        row_count = max(T_count, C_count)

        # row_count 길이의 "새로운 target 블록" 만들기
        new_block = pd.DataFrame(index=range(row_count), columns=target.columns)

        # 1) target 쪽 행을 위부터 복사
        #    (T_count 개만큼)
        if T_count > 0:
            new_block.iloc[:T_count] = tg_df.values

        # 2) conform 값 덮어쓰기
        #    (C_count 개 행에 대해서만 '활용형','부가명칭','부가설명' 갱신)
        for i in range(C_count):  # 0 ~ (C_count-1)
            for col in merge_cols:
                new_block.at[i, col] = cf_df.iloc[i][col]

        merged_list.append(new_block)

    # 모든 ID 그룹별 블록 합치기
    final_df = pd.concat(merged_list, ignore_index=True)
    
    # NaN을 공백 등으로 채우고 싶다면
    final_df.fillna('', inplace=True)

    return final_df

target = merge_target_conform(target, conform)
target['대분류'] = '활용형'
target['대분류 노출 여부'] = 'Yes'
target['중분류 노출 여부'] = 'No'
target['소분류 노출 여부'] = 'No'
target['우선순위'] = int(2)

In [1427]:
LEN = len(target)
for i in range(LEN):
    if target.loc[i, '제공처ID'] == '':
        target.loc[i, '제공처ID'] = target.loc[i-1, '제공처ID']

In [1428]:
def merge_target_related(target: pd.DataFrame, 
                        related: pd.DataFrame) -> pd.DataFrame:
    """
    target과 related를 '제공처ID' 기준으로 병합하되,
    - target에 이미 존재하는 칼럼: ['종류','연관어','연관어표제어ID']를 
        related에서 가져온 값으로 덮어쓴다.
    - target 쪽 행이 더 많으면 남는 행들은 그대로 두고(관련 데이터 없음), 
        related 쪽 행이 더 많으면 초과된 row만큼 새 행을 생성하여 값 채움.
    """

    # 덮어쓸(merge)할 칼럼들
    merge_cols = ["종류", "연관어", "연관어표제어ID"]

    # groupby
    tg_group = target.groupby("제공처ID")
    re_group = related.groupby("제공처ID")

    # 두 DF에 존재하는 모든 제공처ID
    all_ids = sorted(set(target["제공처ID"]) | set(related["제공처ID"]))

    merged_list = []

    for pid in all_ids:
        # (1) 해당 pid 그룹의 target 부분
        if pid in tg_group.groups:
            tg_df = tg_group.get_group(pid).copy()
        else:
            # target에 이 ID가 없다면 빈 df
            tg_df = pd.DataFrame(columns=target.columns)

        # (2) 해당 pid 그룹의 related 부분
        if pid in re_group.groups:
            re_df = re_group.get_group(pid).copy()
        else:
            re_df = pd.DataFrame(columns=related.columns)

        T_count = len(tg_df)
        R_count = len(re_df)

        # 두 그룹 중 더 긴 길이
        row_count = max(T_count, R_count)

        # row_count만큼의 새로운 블록을 만든다 (target 컬럼 구조)
        new_block = pd.DataFrame(index=range(row_count), columns=target.columns, dtype=object)

        # 1) target의 원본값을 위에서부터 복사
        if T_count > 0:
            new_block.iloc[:T_count] = tg_df.values

        # 2) related의 merge_cols 값 덮어쓰기
        #    (R_count가 더 크면 row_count 행 중 R_count행까지는 그 값 채움)
        for i in range(R_count):
            for col in merge_cols:
                new_block.at[i, col] = re_df.iloc[i][col]

        merged_list.append(new_block)

    # 모든 pid 그룹 합쳐 최종 결과
    final_df = pd.concat(merged_list, ignore_index=True)

    # 혹시 NaN을 빈 문자열로 대체하고 싶으면
    final_df.fillna('', inplace=True)

    return final_df

# (3) 함수 호출
target = merge_target_related(target, related)
target['종류(연관어부)'] = target['종류']
target['노출용 연관어'] = target['연관어']
target['연관어(연관어부)'] = target['연관어']
target['노출용 연관어(연관어부)'] = target['연관어']
target['출처'] = 'KOREA'
target['출처(연관단어부)'] = 'KOREA'
target['상세페이지연결여부'] = 'YES'
target['서비스 노출(연관어부)'] = 'YES'
target['검색 노출(연관어부)'] = '통검&사전'
target['상세페이지연결여부(연관단어부)'] = 'YES'
target['서비스 노출(연관단어부)'] = 'YES'
target['검색 노출(연관단어부)'] = '통검&사전'

In [1429]:
target.fillna('', inplace=True)

# 표제어부 중복제거 
condition1 = target['제공처ID'] == ''
target.loc[condition1, '엔트리ID':'검색 노출(표제어부)'] = ''
condition2 = target.duplicated(subset=['제공처ID'], keep='first')
target.loc[condition2, '엔트리ID':'검색 노출(표제어부)'] = ''

# 품사부 중복제거 
condition3 = target['상위품사'].isin(['자동사', '타동사', '형용사'])
target.loc[~condition3, '서비스 노출(품사부)':'검색 노출(품사부)'] = ''

# 뜻풀이부 중복제거 
target['노출용 뜻풀이'] = target['뜻풀이']
LEN = len(target)
for i in range(LEN):
    if target.loc[i, '제공처ID'] == '':
        target.loc[i, '제공처ID'] = target.loc[i-1, '제공처ID']
condition4 = target.duplicated(subset=['제공처ID', '뜻번호', '하위품사', '뜻풀이'], keep='first')
target.loc[condition4, '상위품사':'검색 노출(뜻풀이부)'] = ''
condition4 = target.duplicated(subset=['제공처ID'], keep='first')
target.loc[condition4, '제공처ID'] = ''

# 예문부 중복제거 
target['노출용 예문'] = target['예문']
condition5 = target['예문'] == ''
target.loc[condition5, '예문ID':'검색 노출(예문부)'] = ''

# 연관어부 중복제거 (1)
condition6 = target['연관어'] == ''
target.loc[condition6, '출처':'검색 노출(연관어부)'] = ''

# 활용형 중복제거
condition7 = target['활용형'] == ''
target.loc[condition7, '활용형ID':'부가설명'] = ''

# 연관어부 중복제거 (2)
condition8 = target['연관어(연관어부)'] == ''
target.loc[condition8, '연관단어ID(연관단어부)':'검색 노출(연관단어부)'] = ''

In [1430]:
# def merge_target_related(target: pd.DataFrame, 
#                         related: pd.DataFrame) -> pd.DataFrame:
#     """
#     target과 related를 '제공처ID' 기준으로 병합.
#     target 쪽 '제공처ID'가 여러 줄일 때,
#     related에서 가져올 행 수가 적으면 위부터 붙이고 남는 target 행은 NaN.
#     related의 행 수가 더 많으면 target 행 위에 붙이고 남은 related 행 만큼 
#     새로 행 추가 (target 쪽 컬럼은 NaN).
#     """

#     # 병합할 새 컬럼들
#     merge_cols = ["종류", "연관어", "연관어표제어ID"]

#     # 제공처ID별로 groupby
#     tg_group = target.groupby("제공처ID")
#     re_group = related.groupby("제공처ID")

#     # 모든 제공처ID의 집합
#     all_ids = sorted(set(target["제공처ID"]).union(set(related["제공처ID"])))

#     # 결과를 모을 리스트
#     merged_list = []

#     for pid in all_ids:
#         # target 해당 그룹
#         if pid in tg_group.groups:
#             tg_df = tg_group.get_group(pid)
#         else:
#             # 해당 ID가 없으면 빈 df
#             tg_df = pd.DataFrame(columns=target.columns)

#         # related 해당 그룹
#         if pid in re_group.groups:
#             re_df = re_group.get_group(pid)
#         else:
#             re_df = pd.DataFrame(columns=related.columns)

#         T_count = len(tg_df)
#         R_count = len(re_df)

#         # 두 그룹 중 더 긴 쪽의 길이
#         row_count = max(T_count, R_count)

#         # row_count 크기의 빈 DataFrame을 만들어서 기존 값 복사
#         # (target 쪽)
#         new_tg = pd.DataFrame(index=range(row_count), columns=target.columns)
#         if T_count > 0:
#             new_tg.iloc[:T_count] = tg_df.values
        
#         # (related 쪽에서 가져올 열)
#         new_re = pd.DataFrame(index=range(row_count), columns=merge_cols)
#         if R_count > 0:
#             new_re.iloc[:R_count] = re_df[merge_cols].values

#         # 나란히 붙이기
#         merged_df = pd.concat([new_tg.reset_index(drop=True),
#                                 new_re.reset_index(drop=True)], axis=1)
#         merged_list.append(merged_df)

#     # 모든 ID 그룹을 다시 합침
#     final_df = pd.concat(merged_list, ignore_index=True)
#     return final_df

# target = merge_target_related(target, related)

In [1431]:
# target = target.fillna('')
# # 중복제거 
# condition1 = target.duplicated(subset=['제공처ID', '뜻번호', '하위품사'], keep='first')
# # 패턴정보까지 제거 
# condition2 = target.duplicated(subset=['제공처ID'], keep='first')
# # 뜻번호 문제인 애들
# condition3 = target.duplicated(subset=['제공처ID', '어깨번호', '형태', '원어', '어원'], keep='first')

# col_range1 = target.loc[:, '제공처ID':'패턴정보'].columns
# col_range2 = target.loc[:, '제공처ID':'어원'].columns
# col_range3 = target.loc[:, '제공처ID':'어원'].columns

# target.loc[condition1, col_range1] = ''
# target.loc[condition2, col_range2] = ''
# target.loc[condition3, col_range3] = ''

In [1432]:
# 아이디 매칭표 불러오기
NAVER = pd.read_csv(id_path)
NAVER.fillna('', inplace=True)
NAVER.columns

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/2600345270.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  NAVER = pd.read_csv(id_path)
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/2600345270.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  NAVER.fillna('', inplace=True)


Index(['엔트리ID', '표제어', '어깨번호', '표제어종류', '제공처ID', '편집자', '원어 멤버ID', '차별 표현',
       '차별 표현 링크', '뜻풀이 ID', '예문 ID', '활용형 ID', '연관단어 ID', '연관어 뜻 ID',
       '연관어 표제어 ID', '연관단어 ID.1', '연관어'],
      dtype='object')

In [1433]:
# 엔트리ID, 원어맴버ID 매칭
# (1) 제공처ID가 ''이 아닌 행만 추출 (슬라이스)
target_no_empty = target[['엔트리ID', '제공처ID', '원어맴버ID']]
target_no_empty = target_no_empty[target_no_empty['제공처ID'] != ''].copy()

# (2) 슬라이스의 인덱스를 보존하기 위해 reset_index로 'index' 컬럼화
target_no_empty.reset_index(inplace=True)

# (3) NAVER와 left join 수행 (제공처ID 기준)
merged_no_empty = target_no_empty.merge(
    NAVER[['제공처ID', '엔트리ID', '원어 멤버ID']], 
    how='left', 
    on='제공처ID',
    suffixes=('_original', '')  # 왼쪽 df에 _original, 오른쪽 df(=NAVER)에 접미사 없음
)
# # (4) merge된 결과(merged_no_empty)에 담긴 '엔트리ID', '원어 멤버ID'를
# #     원본 target에 덮어쓰기
# target.loc[merged_no_empty['index'], '엔트리ID'] = merged_no_empty['엔트리ID']
# target.loc[merged_no_empty['index'], '원어맴버ID'] = merged_no_empty['원어 멤버ID']
for i, index in enumerate(list(merged_no_empty['index'])):
    target.loc[index, '엔트리ID'] = list(merged_no_empty['엔트리ID'])[i]
    target.loc[index, '원어맴버ID'] = list(merged_no_empty['원어 멤버ID'])[i]

In [1434]:
NAVER = pd.read_csv(id_mean_path)
NAVER.fillna('', inplace=True)
NAVER.columns

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/2523782737.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  NAVER = pd.read_csv(id_mean_path)
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/2523782737.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  NAVER.fillna('', inplace=True)


Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '뜻풀이번호', '차별 표현',
       '차별 표현 링크', '뜻풀이 ID'],
      dtype='object')

In [1435]:
# 차별표현, 차별표현링크, 뜻풀이부 
LEN = len(target)
target['가짜뜻번호'] = ''
index = 2
for i in range(LEN):
    if target.loc[i, '제공처ID'] == '':
        if target.loc[i, '하위품사'] != '':
            target.loc[i, '가짜뜻번호'] = index 
            index += 1
    else:
        target.loc[i, '가짜뜻번호'] = int(1) 
        index = 2
for i in range(LEN):
    if target.loc[i, '엔트리ID'] == '':
        target.loc[i, '엔트리ID'] = target.loc[i-1, '엔트리ID']

# NAVER에서 (엔트리ID, 뜻풀이번호)를 키로 하고, (차별 표현, 차별 표현 링크, 뜻풀이 ID)를 값으로 하는 dict 생성
naver_dict = {}
for i, row in NAVER.iterrows():
    if row['엔트리ID'] != '':
        temp = row['엔트리ID']
    if row['엔트리ID'] == '':
        row['엔트리ID'] = temp
    key = (row['엔트리ID'], row['뜻풀이번호'])
    val = (row['차별 표현'], row['차별 표현 링크'], row['뜻풀이 ID'])
    naver_dict[key] = naver_dict.get(key, []) + [val]

# 이때 한 엔트리ID+뜻풀이번호 조합에 여러 행이 있을 수도 있다면, list를 append할 수도 있음
# 이후 target에서 하나씩 꺼내서 매핑 가능

def get_naver_values(row):
    # row에서 (엔트리ID, 뜻번호) 추출
    key = (row['엔트리ID'], row['가짜뜻번호'])
    if key in naver_dict:
        # NAVER 값이 여러 개 있으면 첫 번째만 쓰거나, 필요에 따라 처리
        return naver_dict[key][0]  # (차별 표현, 차별 표현 링크, 뜻풀이 ID)
    else:
        return (None, None, None)

vals = target.apply(get_naver_values, axis=1)
target[['차별표현', '차별표현 링크', '뜻풀이ID']] = pd.DataFrame(vals.tolist(), index=target.index)
target.drop(columns=['가짜뜻번호'], inplace=True)
condition = target.duplicated(subset=['엔트리ID'], keep='first')
target.loc[condition, '엔트리ID'] = ''

In [1436]:
# 예문 ID 매칭
NAVER = pd.read_csv(id_exam_path)
NAVER.fillna('', inplace=True)
NAVER.columns

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/1690455174.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  NAVER.fillna('', inplace=True)


Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '예문번호', '예문 ID'], dtype='object')

In [1437]:
LEN = len(target)
target['예문번호'] = ''
index = 1
for i in range(LEN):
    if target.loc[i, '제공처ID'] == '':
        if target.loc[i, '예문'] != '':
            target.loc[i, '예문번호'] = index 
            index += 1
    else:
        index = 1
        if target.loc[i, '예문'] != '':
            target.loc[i, '예문번호'] = index 
            index += 1
for i in range(LEN):
    if target.loc[i, '엔트리ID'] == '':
        target.loc[i, '엔트리ID'] = target.loc[i-1, '엔트리ID']

naver_dict = {}
for i, row in NAVER.iterrows():
    if row['엔트리ID'] != '':
        temp = row['엔트리ID']
    if row['엔트리ID'] == '':
        row['엔트리ID'] = temp
    key = (row['엔트리ID'], row['예문번호'])
    val = (row['예문 ID'])
    naver_dict[key] = naver_dict.get(key, []) + [val]

def get_naver_values(row):
    key = (row['엔트리ID'], row['예문번호'])
    if key in naver_dict:
        # NAVER 값이 여러 개 있으면 첫 번째만 쓰거나, 필요에 따라 처리
        return naver_dict[key][0]  # (예문 ID)
    else:
        return (None)

vals = target.apply(get_naver_values, axis=1)
target[['예문ID']] = pd.DataFrame(vals.tolist(), index=target.index)
target.drop(columns=['예문번호'], inplace=True)
condition = target.duplicated(subset=['엔트리ID'], keep='first')
target.loc[condition, '엔트리ID'] = ''

In [1438]:
# 활용형 ID 매칭
NAVER = pd.read_csv(id_use_path)
NAVER.fillna('', inplace=True)
NAVER.columns

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/810149915.py:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  NAVER = pd.read_csv(id_use_path)
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/810149915.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  NAVER.fillna('', inplace=True)


Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '활용번호', '활용형 ID'], dtype='object')

In [1439]:
LEN = len(target)
target['활용번호'] = ''
index = 1
for i in range(LEN):
    if target.loc[i, '제공처ID'] == '':
        if target.loc[i, '대분류'] != '':
            target.loc[i, '활용번호'] = index 
            index += 1
    else:
        index = 1
        if target.loc[i, '대분류'] != '':
            target.loc[i, '활용번호'] = index 
            index += 1
for i in range(LEN):
    if target.loc[i, '엔트리ID'] == '':
        target.loc[i, '엔트리ID'] = target.loc[i-1, '엔트리ID']

naver_dict = {}
for i, row in NAVER.iterrows():
    if row['엔트리ID'] != '':
        temp = row['엔트리ID']
    if row['엔트리ID'] == '':
        row['엔트리ID'] = temp
    key = (row['엔트리ID'], row['활용번호'])
    val = (row['활용형 ID'])
    naver_dict[key] = naver_dict.get(key, []) + [val]

def get_naver_values(row):
    key = (row['엔트리ID'], row['활용번호'])
    if key in naver_dict:
        # NAVER 값이 여러 개 있으면 첫 번째만 쓰거나, 필요에 따라 처리
        return naver_dict[key][0]  # (예문 ID)
    else:
        return (None)

vals = target.apply(get_naver_values, axis=1)
target[['활용형ID']] = pd.DataFrame(vals.tolist(), index=target.index)
target.drop(columns=['활용번호'], inplace=True)
condition = target.duplicated(subset=['엔트리ID'], keep='first')
target.loc[condition, '엔트리ID'] = ''

In [1440]:
# 연관단어 ID 매칭
NAVER = pd.read_csv(id_rel_path)
NAVER.fillna('', inplace=True)
NAVER.columns

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_51381/1570972607.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  NAVER.fillna('', inplace=True)


Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '연관어', '연관단어 ID',
       '연관어 뜻 ID', '연관단어 ID.1', '연관어 표제어 ID'],
      dtype='object')

In [1441]:
LEN = len(target)

# 1) target에서 엔트리ID가 빈('')이면 바로 윗행 엔트리ID로 채우기
#    (주의: i=0일 때 i-1은 -1이므로 로직 검증 필요)
for i in range(LEN):
    if target.loc[i, '엔트리ID'] == '':
        target.loc[i, '엔트리ID'] = target.loc[i-1, '엔트리ID']

# 2) NAVER에서 (엔트리ID, 연관어) -> 여러 개의 값 list로 저장
naver_dict = {}
for i, row in NAVER.iterrows():
    if row['엔트리ID'] != '':
        temp = row['엔트리ID']
    if row['엔트리ID'] == '':
        row['엔트리ID'] = temp
    key = (row['엔트리ID'], row['연관어'])
    val = (
        row['연관단어 ID'],
        row['연관어 뜻 ID'],
        row['연관어 표제어 ID'],
        'KOREA',
        'KOREA',
        row['연관단어 ID.1'],
        row['연관어 표제어 ID']
    )
    naver_dict[key] = naver_dict.get(key, []) + [val]

# 3) get_naver_values 함수: pop(0)로 한 번 사용한 값은 제거
def get_naver_values(row):
    key = (row['엔트리ID'], row['연관어'])
    if key in naver_dict and len(naver_dict[key]) > 0:
        return naver_dict[key].pop(0)
    else:
        return ('', '', row['연관어표제어ID'], '', '', '', row['연관어표제어ID'])

vals = target.apply(get_naver_values, axis=1)

# 4) 반환값(5개 튜플)을 각 컬럼에 할당
target[['연관단어ID', '연관어뜻ID', '연관어표제어ID', '출처', '출처(연관단어부)', '연관단어ID(연관단어부)', '연관어표제어ID(연관어부)']] = \
    pd.DataFrame(vals.tolist(), index=target.index)

# 5) 중복된 엔트리ID 처리는 원하는 대로
condition = target.duplicated(subset=['엔트리ID'], keep='first')
target.loc[condition, '엔트리ID'] = ''


In [1442]:
NAVER = pd.read_csv(id_present_path)
NAVER = NAVER[['엔트리ID', '제공처ID']]
NAVER.fillna('', inplace=True)
NAVER = NAVER[(NAVER != '').any(axis=1)]
NAVER

,엔트리ID,제공처ID
0,000ddcb800734ecf940f0e26371de191,46
1,0b52f545b2f7494c9a9352b3ddbef669,52
2,0b83a85e13834d749238a24e36bf1b51,74
3,0cdd5cf3e6814d72bb409739c8ea6025,38
4,10fb541bb3904308bbccbe205a110ea8,22
...,...,...
400321,f1a528b9023a42a3ae583b5205071cfd,466341
400322,f2dda3e8636846fea325b696400cce1c,466304
400323,f3ddf94a04414907a1e886fb638ece42,466289
400324,f9d321279e8b4085bb38d4af7f47e5b8,466344


In [1443]:
# # 연관어 ID 매칭
# # 1) ''(빈 문자열)을 NaN으로 변환
# target["연관어표제어ID"] = target["연관어표제어ID"].replace('', pd.NA)

# # 2) NAVER 딕셔너리 만들기: {제공처ID → 엔트리ID}
# naver_dict = NAVER.set_index("제공처ID")["엔트리ID"].to_dict()

# # 3) '연관어표제어ID'가 NaN이 아닌 경우에만 대체
# def replace_with_entry_id(x):
#     # x가 NaN이면 건너뛰기
#     if pd.isna(x):
#         return x
#     # NAVER에 해당 키가 있으면 치환, 없으면 원래 값 유지
#     return naver_dict.get(x, x)

# target["연관어표제어ID"] = target["연관어표제어ID"].apply(replace_with_entry_id)
# target = target.fillna('')
condition1 = target['연관어'] == ''
target.loc[condition1, '출처':'검색 노출(연관어부)'] = ''
target.loc[condition1, '연관단어ID(연관단어부)':'검색 노출(연관단어부)'] = ''

LEN = len(target)
for i in range(LEN):
    if target.loc[i, '연관어'] != '':
        if target.loc[i, '연관단어ID'] == '':
            print(target.loc[i, '연관어'])
            matched_int = int(target.loc[i, '연관어표제어ID'])
            matched_row = NAVER[NAVER['제공처ID'] == matched_int]
            if not matched_row.empty:
                # 매칭된 행이 최소 1개라도 있으면 첫 번째 행을 사용
                naver_id = matched_row.iloc[0]['엔트리ID']
                print(naver_id)
                target.loc[i, '연관어표제어ID'] = naver_id
                target.loc[i, '연관어표제어ID(연관어부)'] = naver_id
            else:
                # 매칭된 행이 전혀 없을 때 처리 (로그 표시 or 그냥 건너뛰기 등)
                print(f"엔트리ID를 찾지 못했습니다. 제공처ID={matched_int} (행 index={i})")
                # 필요하다면 아래처럼 빈 칸으로 두거나, 다른 기본값을 넣을 수도 있음
                target.loc[i, '연관어표제어ID'] = ''
                target.loc[i, '연관어표제어ID(연관어부)'] = ''

편죽
25c3728eee5c4383903ca875c35f430f
편축
771c7eda84f04c5da730fea3aa4ef63e
변축
71929de5ac544899824fde70987289c3
말라야인
2863a50fd4ce4fab8d2e7c526b8b3485
말레이인
550a2980f16a4cd7a31f3de4f9e45925
파구
c432faa737fb42cda23468087b73ff0d
호구별성
07facc16669a4312b1d3d97b1e2570a1
호귀별성
fd01a61a7f80412abf62466945539853
별신
d8bcd5605033450a8723fc0ca26446a2
강남별성
f7c651b43cb74d72931198027483b5d7
이알피
230bb31313ab411a84a4b7cd43c79b09
마실돌이
d96a2fa3e322420695a7f66710f7a812
의사
9298e99e0ddf4f01b68f373c4724c244
의향
29a9243b89764cb6b281442fe5331ef1
마음씨
c6027b454a7d47218ec9fa7503840d12
맘
c9ae4ff064d44fa996ac08b4ff2a0f3b
속마음
55a1218b5073432e956d9ddd72b8a6b9
심중
a3ba12fdb92b4a2db8f9a2cd1a6fe69e
맘
c9ae4ff064d44fa996ac08b4ff2a0f3b
맘
c9ae4ff064d44fa996ac08b4ff2a0f3b
심근
acd3c974e1cb4fdc8947f8cd81b703ce
속심
1695dc149865461d9c5ad20490f33e42
내심
fe577f30ab85412295881af573cdd4a2
내의
c06315058afe4c7db9dedf05d3f99956
저의
ca7f6bb1201642f3877e69f7e9b596b6
흉금
f3a97a9df1024017aee14690aab43f0e
흉심
6d4ab81a598f4a7dbfdc5ff40748d813
염화 석회
7f73ea52da

In [1444]:
con = target['연관어'] == '방관'
target[con]

,엔트리ID,집필 여부,표제어,어깨번호,집필상태,편집자,최초 등록 시간,마지막 편집 시간,표제어종류,제공처ID,...,연관단어ID(연관단어부),종류(연관어부),연관어(연관어부),노출용 연관어(연관어부),참고 문법(연관단어부),출처(연관단어부),연관어표제어ID(연관어부),상세페이지연결여부(연관단어부),서비스 노출(연관단어부),검색 노출(연관단어부)
63736,,,,,,,,,,,...,47c5ba4ac4ab478fbe8334cc28447ad1,유의,방관,방관,,KOREA,a592d649da6d458991f4cf3c56d1eb75,YES,YES,통검&사전


In [1445]:
# 뜻풀이 EID 변경
LEN = len(target)
for i in range(LEN):
    if 'EID' in target.loc[i, '노출용 뜻풀이']:
        matches = re.findall(r'(\S+)&EID=(\d+);', target.loc[i, '노출용 뜻풀이'])
        for word, match in matches:
            matched_int = int(match)
            matched_row = NAVER[NAVER['제공처ID'] == matched_int]
            if not matched_row.empty:
                naver_id = matched_row['엔트리ID'].iloc[0]
                template = f'<a class="direct" href="https://ko.dict.naver.com/#/entry/koko/{naver_id}">{word}</a>'
                pattern_to_replace = f'{word}&EID={match};'
                target.loc[i, '노출용 뜻풀이'] = target.loc[i, '노출용 뜻풀이'].replace(pattern_to_replace, template)
            else:
                print("NOT MATCHED")

NOT MATCHED
NOT MATCHED


In [1446]:
target.to_csv(output_file_path, index=False)

In [1447]:
print(f"time spend : {time() - start_time}")

time spend : 744.4643797874451
